In [1]:
import ray
import pandas as pd


### Import Methods

#### Way 1

without directly using the
`from .predictability.utils import get_column_combinations`
in `src/asd/__init__.py` file.

In [ ]:
'''
import asd.predictability.utils as asdpu
asdpu.get_column_combinations(all_cols=[1, 2, 3, 4, 5, 6],
                                  inputs=4,
                                  outputs=1,
                                  targets=[5, 6]
                                  )
'''

#### Way 2

with using the import
`from .predictability.utils import get_column_combinations`
in `src/asd/__init__.py` file.

In [ ]:
import asd

In [2]:
asd.get_column_combinations(all_cols=[1, 2, 3, 4, 5, 6],
                                  inputs=4,
                                  outputs=1,
                                  targets=[5, 6]
                                  )

[(1, 2, 3, 4, 5), (1, 2, 3, 4, 6)]

In [5]:
ray.init()

Python version:,3.9.5
Ray version:,2.0.1


# Basic Example

## Load Data

Basic data that can be understood as two different input columns (linear and random values) and three different output columns (sin of linear x, 1 + sin of linear x and random values).

In [6]:
data = pd.read_csv("example_data.csv", dtype=float)

In [7]:
data.head()

,lin_x,rand_x,sinx,randomy,sinx_plus1
0,1.0,-0.55,0.099833,0.09,1.099833
1,2.0,-0.44,0.198669,0.83,1.198669
2,3.0,0.94,0.295520,0.12,1.295520
3,4.0,-0.61,0.389418,0.41,1.389418
4,5.0,-0.51,0.479426,-0.70,1.479426


## Get Combinations function

This function can be used to determine the overall number of combinations the predictability routine analyses given the number of data columns, fitting type etc. This allows to estimate the overall runtime of the predictability routine.

The function returns a list of combination tuples, where the first inputs-many elements correspond to the inputs and the remaining ouputs-many to the targets. The argument targets can be used to define columns that should exclusively be regarded as targets.

In [8]:
# applied to a numerical example:
asd.get_column_combinations(all_cols=[1, 2, 3, 4, 5, 6],
                                  inputs=3,
                                  outputs=1,
                                  targets=[5, 6]
                                  )

[(1, 2, 3, 5),
 (1, 2, 3, 6),
 (1, 2, 4, 5),
 (1, 2, 4, 6),
 (1, 3, 4, 5),
 (1, 3, 4, 6),
 (2, 3, 4, 5),
 (2, 3, 4, 6)]

In [9]:
# the argument "amount_only" can be used to output the amout of combinations only
asd.get_column_combinations(all_cols=[1, 2, 3, 4, 5, 6],
                                  inputs=3,
                                  outputs=1,
                                  targets=[5, 6],
                                  amount_only=True
                                  )

8

In [10]:
# applied to the data above
print(asd.get_column_combinations(all_cols=data.columns, inputs=1, outputs=1, targets=["sinx", "randomy", "sinx_plus1"]))

[('lin_x', 'sinx'), ('lin_x', 'randomy'), ('lin_x', 'sinx_plus1'), ('rand_x', 'sinx'), ('rand_x', 'randomy'), ('rand_x', 'sinx_plus1')]


In [11]:
print(asd.get_column_combinations(all_cols=data.columns, inputs=1, outputs=1, targets=["sinx", "randomy", "sinx_plus1"],
                                  amount_only=True))

6


## Predictability function

Running the predictability function over all possible 1+1 combinations where the respective target is either sin, sin+1 or random and the input is linear x or random values.

The purpose of having sin + 1 is to have a tuple that exclusively contains positive values so fitting a power law can be applied.

### "normal" routine

In [ ]:
metrics_dict, datas_dict = asd.run_predictability(data=data,
                                          input_cols=1,
                                          output_cols=1,
                                          col_set=None,
                                          targets=["sinx", "randomy", "sinx_plus1"],
                                          method="kNN",
                                          random_state_split=None,
                                          refined_n_best=0
                                          )

_Note that starting the Ray instance accounts for most of the runtime. Start the cell above again and runtime will be ~1s._

### greedy routine

In [ ]:
greedy_metrics_dict, greedy_datas_dict = asd.run_predictability(data=data,
                                          input_cols=2,
                                          output_cols=1,
                                          col_set=None,
                                          targets=["sinx", "randomy", "sinx_plus1"],
                                          method="kNN",
                                          random_state_split=None,
                                          greedy=True,
                                          refined_n_best=0
                                          )

### Results

In [ ]:
pd.DataFrame.from_dict(metrics_dict).transpose()

In [ ]:
pd.DataFrame.from_dict(greedy_metrics_dict).transpose()

### Structure of returned dictionaries

In [ ]:
struc_dict = datas_dict[list(datas_dict.keys())[0]]
for key in list(struc_dict.keys()):
    if type(struc_dict[key]) is dict:
        print(key, "\t dict with key(s):\t", list(struc_dict[key].keys()))
    else:
        print(key, "\t type:\t", type(struc_dict[key]), "\t shape:\t", struc_dict[key].shape)

### Plotting

In [ ]:
asd.predictability_plot(datas_dict, list(datas_dict.keys())[0], plot_along=["linear", "mean"])

## Tuple Selection function

This function can be used to limit the number of tuples that is further analysed in more detail.

In [ ]:
selected_tuples = asd.predictability.core.tuple_selection(metrics_dict, n_best=2)
selected_tuples

## Refine Predictability function

This function is used to further refine the predictability of the previously selected best tuples.

In [ ]:
refined_metrics_dict, refined_datas_dict = asd.refine_predictability(best_tuples=selected_tuples,
                                                                 data_dict=datas_dict,
                                                                 time_left_for_this_task=60,
                                                                 n_jobs=-1,
                                                                 use_ray=True,
                                                                 )

In [ ]:
pd.DataFrame.from_dict(refined_metrics_dict).transpose()

#### Structure of returned dictionaries

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
struc_dict = refined_datas_dict[list(refined_datas_dict.keys())[0]]
for key in list(struc_dict.keys()):
    if type(struc_dict[key]) is dict:
        print(key, "\t dict with key(s):\t"#, list(struc_dict[key].keys())
              )
    elif type(struc_dict[key]) is Pipeline:
        print(key, "\t type:\t", type(struc_dict[key]))
    else:
        print(key, "\t type:\t", type(struc_dict[key]), "\t shape:\t", struc_dict[key].shape)

#### Plotting

In [ ]:
asd.predictability_plot(refined_datas_dict,
                        list(refined_datas_dict.keys())[0],
                        refined_dict=True,
                        refined_input_datas_dict=datas_dict,
                        plot_along=[#"linear", "mean",
                                    "init"]
                        )